# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [1]:
# pip install pyowm 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [36]:
import requests
import pyowm
import json
from auth import get_key

api_key = get_key()

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [37]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key,
    'units' : "metric"
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 18.96,
  'feels_like': 19.13,
  'temp_min': 17.79,
  'temp_max': 20,
  'pressure': 995,
  'humidity': 85,
  'sea_level': 995,
  'grnd_level': 991},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 210, 'gust': 12.35},
 'rain': {'1h': 0.49},
 'clouds': {'all': 75},
 'dt': 1756901862,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1756876593,
  'sunset': 1756925015},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [40]:
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [69]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 18.51°C, Humidity: 86%, Wind Speed: 6.17 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [70]:
mgr = owm.weather_manager()
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
temperature = observation.weather.temperature("celsius")["temp"]
humidity = observation.weather.humidity
wind_speed = observation.weather.wind()["speed"]

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 18.19°C, Humidity: 88%, Wind Speed: 5.81 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [71]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [72]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-09-03 15:00:00, Temp: 19.53°C
Time: 2025-09-03 18:00:00, Temp: 22.86°C
Time: 2025-09-03 21:00:00, Temp: 24.04°C
Time: 2025-09-04 00:00:00, Temp: 20.7°C
Time: 2025-09-04 03:00:00, Temp: 18.34°C
Time: 2025-09-04 06:00:00, Temp: 17.12°C
Time: 2025-09-04 09:00:00, Temp: 19.08°C
Time: 2025-09-04 12:00:00, Temp: 19.89°C
Time: 2025-09-04 15:00:00, Temp: 23.48°C
Time: 2025-09-04 18:00:00, Temp: 25.01°C
Time: 2025-09-04 21:00:00, Temp: 23.39°C
Time: 2025-09-05 00:00:00, Temp: 22.71°C
Time: 2025-09-05 03:00:00, Temp: 22.36°C
Time: 2025-09-05 06:00:00, Temp: 19.95°C
Time: 2025-09-05 09:00:00, Temp: 20.23°C
Time: 2025-09-05 12:00:00, Temp: 20.53°C
Time: 2025-09-05 15:00:00, Temp: 24.22°C
Time: 2025-09-05 18:00:00, Temp: 27.24°C
Time: 2025-09-05 21:00:00, Temp: 27.91°C
Time: 2025-09-06 00:00:00, Temp: 26.9°C
Time: 2025-09-06 03:00:00, Temp: 24.65°C
Time: 2025-09-06 06:00:00, Temp: 24°C
Time: 2025-09-06 09:00:00, Temp: 23.18°C
Time: 2025-09-06 12:00:00, Temp: 23.4°C
Time: 2025-09-06 15:00

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [86]:
three_h_forecast = mgr.forecast_at_place('New York,US', '3h').forecast
for weather in three_h_forecast:
    dt = weather.reference_time('iso')
    temp = weather.temperature("celsius")["temp"]
    print(f"Time: {dt}, Temp: {temp}°C")

Time: 2025-09-03 15:00:00+00:00, Temp: 19.8°C
Time: 2025-09-03 18:00:00+00:00, Temp: 22.96°C
Time: 2025-09-03 21:00:00+00:00, Temp: 24.04°C
Time: 2025-09-04 00:00:00+00:00, Temp: 20.7°C
Time: 2025-09-04 03:00:00+00:00, Temp: 18.34°C
Time: 2025-09-04 06:00:00+00:00, Temp: 17.12°C
Time: 2025-09-04 09:00:00+00:00, Temp: 19.08°C
Time: 2025-09-04 12:00:00+00:00, Temp: 19.89°C
Time: 2025-09-04 15:00:00+00:00, Temp: 23.48°C
Time: 2025-09-04 18:00:00+00:00, Temp: 25.01°C
Time: 2025-09-04 21:00:00+00:00, Temp: 23.89°C
Time: 2025-09-05 00:00:00+00:00, Temp: 22.59°C
Time: 2025-09-05 03:00:00+00:00, Temp: 22.39°C
Time: 2025-09-05 06:00:00+00:00, Temp: 20.23°C
Time: 2025-09-05 09:00:00+00:00, Temp: 20.42°C
Time: 2025-09-05 12:00:00+00:00, Temp: 20.81°C
Time: 2025-09-05 15:00:00+00:00, Temp: 24.27°C
Time: 2025-09-05 18:00:00+00:00, Temp: 26.84°C
Time: 2025-09-05 21:00:00+00:00, Temp: 27.72°C
Time: 2025-09-06 00:00:00+00:00, Temp: 26.04°C
Time: 2025-09-06 03:00:00+00:00, Temp: 24.54°C
Time: 2025-09-0

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [89]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 291.35°C, Fahrenheit: 556.4300000000001°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [90]:
mgr = owm.weather_manager()
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
temperature_celsius = observation.weather.temperature("celsius")["temp"]
temperature_fahrenheit = observation.weather.temperature("fahrenheit")["temp"]

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 18.51°C, Fahrenheit: 65.32°F
